#Resume Screener

In [21]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.activations import tanh
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import tensorflow.keras.layers as layers
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier

#Helper Function
def cleanResume(resumeText):
    resumeText = re.sub('httpS+s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^x00-x7f]',r' ', resumeText) 
    resumeText = re.sub('s+', ' ', resumeText)  # remove extra whitespace
    return resumeText

## Data Load and Pre processing

In [ ]:
#Loading Data
data_path='/content/drive/MyDrive/Colab Notebooks/Resume Recommendation/UpdatedResumeDataSet.csv'
df = pd.read_csv(data_path)

#Pre preprocessing
#Regex
df['cleaned_resume'] = df.Resume.apply(lambda x: cleanResume(x))

#Encoding labels
var_mod = ['Category']
classes = df['Category'].unique()
le = LabelEncoder()
for i in var_mod:
    df[i] = le.fit_transform(df[i])

#Vectorization
requiredText = df['cleaned_resume'].values
requiredTarget = df['Category'].values
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    stop_words='english',
    max_features=1500)
word_vectorizer.fit(requiredText)
WordFeatures = word_vectorizer.transform(requiredText)

#Splitting the data
X_train,X_test,y_train,y_test = train_test_split(WordFeatures,requiredTarget,random_state=0, test_size=0.2)

##Model

In [22]:
#Train and Fit
clf = OneVsRestClassifier(KNeighborsClassifier())
clf.fit(X_train, y_train)

#Accuracy Score
print('Accuracy of KNeighbors Classifier on training set: {:.2f}'.format(clf.score(X_train, y_train)))
print('Accuracy of KNeighbors Classifier on test set: {:.2f}'.format(clf.score(X_test, y_test)))

#Prediction of class (Job)
prediction = clf.predict(X_test)
for preds in prediction:
  print(classes[preds])